In [3]:
##предварительная обработка текста

In [29]:
#!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 536.2 kB/s eta 0:00:001m520.4 kB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 2.3 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=caa31786ada9b716352a5abf36be1db5e8e9b628eb5a9c0c2cabcd11da0a3f6a
  Stored in directory: /home/andrey/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
import numpy as np
import pandas as pd
import re
import nltk
import string

In [4]:
#help(pd.read_csv)

In [5]:
##Чтение датасета с текстом
df = pd.read_csv("reviews.csv", encoding='UTF8', sep="\t")
df.head(10)

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
5,Очень глубокие проймы,negative
6,Я недовольна заказом.Я вот одного не понимаю п...,negative
7,"заказала размер s на от 64,об 94,начнем с того...",negative
8,Заказ я сделала в июле. С тех пор посылка отсл...,negative
9,Ужасное качество товара!,negative


In [8]:
df.iloc[[1]]
df.tail(3)

,review,sentiment
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive
89999,хорошая куртка. постороннего запаха нет. швы р...,positive


In [9]:
##Нижний регистр. Например, Товар --> товар, УРА --> ура
df['text_processed']  = df['review'].str.lower()
df['text_processed'].head()

0    качество плохое пошив ужасный (горловина напер...
1    товар отдали другому человеку, я не получила п...
2    ужасная синтетика! тонкая, ничего общего с пре...
3    товар не пришел, продавец продлил защиту без м...
4        кофточка голая синтетика, носить не возможно.
Name: text_processed, dtype: object

In [10]:
df.head()

,review,sentiment,text_processed
0,качество плохое пошив ужасный (горловина напер...,negative,качество плохое пошив ужасный (горловина напер...
1,"Товар отдали другому человеку, я не получила п...",negative,"товар отдали другому человеку, я не получила п..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"ужасная синтетика! тонкая, ничего общего с пре..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"товар не пришел, продавец продлил защиту без м..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"кофточка голая синтетика, носить не возможно."


In [11]:
#Удаление пунктуации
df['text_processed'] = df['text_processed'].str.replace('[^\w\s]','')
df['text_processed'].head()

0    качество плохое пошив ужасный (горловина напер...
1    товар отдали другому человеку, я не получила п...
2    ужасная синтетика! тонкая, ничего общего с пре...
3    товар не пришел, продавец продлил защиту без м...
4        кофточка голая синтетика, носить не возможно.
Name: text_processed, dtype: object

In [13]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/andrey/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [18]:
#Удаление стоп-слов
#Импорт стоп-слов из бибилиотеки nltk

#nltk.download("stopwords")
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('russian'))
# функция, удаляющая стопслова из текстов

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
# Применим функцию удаления стоп-слов
df['text_processed'] = df['text_processed'].apply(stopwords)
df['text_processed'].head()

0    качество плохое пошив ужасный (горловина напер...
1    товар отдали другому человеку, получила посылк...
2    ужасная синтетика! тонкая, общего представленн...
3    товар пришел, продавец продлил защиту моего со...
4           кофточка голая синтетика, носить возможно.
Name: text_processed, dtype: object

In [20]:
#Удаление высокочастотных слов
from collections import Counter
cnt = Counter()
for text in df['text_processed'].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('очень', 26571),
 ('размер', 16715),
 ('товар', 11381),
 ('продавец', 11095),
 ('качество', 10909),
 ('деньги', 9323),
 ('это', 8004),
 (',', 7945),
 ('заказ', 6783),
 ('-', 6536)]

In [21]:
#ужаление часто встречающиеся слов в корпусе при помощи tf-idf
# Удаление самых частотных слов
freq = set([w for (w, wc) in cnt.most_common(10)])
# функция удаления слов
def freqwords(text):
    return " ".join([word for word in str(text).split() if word not 
in freq])
# применение функции
df['text_processed'] = df['text_processed'].apply(freqwords)
df['text_processed'].head()

0    плохое пошив ужасный (горловина наперекос) фот...
1    отдали другому человеку, получила посылку. лад...
2    ужасная синтетика! тонкая, общего представленн...
3    пришел, продлил защиту моего согласия. продавц...
4           кофточка голая синтетика, носить возможно.
Name: text_processed, dtype: object

In [22]:
#Удаление низкочастотных слов
freq = pd.Series(' '.join(df['text_processed']).split()).value_counts()[-10:] # 10 rare words
freq = list(freq.index)
df['text_processed'] = df['text_processed'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['text_processed'].head()

0    плохое пошив ужасный (горловина наперекос) фот...
1    отдали другому человеку, получила посылку. лад...
2    ужасная синтетика! тонкая, общего представленн...
3    пришел, продлил защиту моего согласия. продавц...
4           кофточка голая синтетика, носить возможно.
Name: text_processed, dtype: object

In [23]:
# Удаление эмодзи
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
#passing the emoji function to 'text_rare'
df['text_processed'] = df['text_processed'].apply(emoji)

In [24]:
#Удаление цифр
df['text_processed'] = df['text_processed'].str.replace('\d+', '') 
df['text_processed'].head()

0    плохое пошив ужасный (горловина наперекос) фот...
1    отдали другому человеку, получила посылку. лад...
2    ужасная синтетика! тонкая, общего представленн...
3    пришел, продлил защиту моего согласия. продавц...
4           кофточка голая синтетика, носить возможно.
Name: text_processed, dtype: object

In [25]:
#Удаление URL
# Function for url's
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)
# Examples
text = "This is my website, https://www.link.com"
remove_urls(text)
#Passing the function to 'text_rare'
df['text_processed'] = df['text_processed'].apply(remove_urls)

In [26]:
#Удаление HTML-тегов
from bs4 import BeautifulSoup
#Function for removing html
def html(text):
    return BeautifulSoup(text, "lxml").text
# Examples
text = """<div>
<h1> Это </h1>
<p> странца</p>
<a href="https://www.link.com/"> пример</a>
</div>
"""
print(html(text))
# Passing the function to 'text_rare'
df['text_processed'] = df['text_processed'].apply(html)


 Это 
 странца
 пример




/usr/lib/python3/dist-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(
/usr/lib/python3/dist-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: ".." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(
/usr/lib/python3/dist-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: "///" looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(
/usr/lib/python3/dist-packages/bs4/__init__.py:337: MarkupResemblesLocatorWarning: "/" looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [27]:
#Токенизация
def tokenization(text):
    text = re.split('\W+', text)
    return text
# Passing the function to 'text_rare' and store into'text_token'
df['text_processed'] = df['text_processed'].apply(lambda x: tokenization(x.lower()))
df[['text_processed']].head()

,text_processed
0,"[плохое, пошив, ужасный, горловина, наперекос,..."
1,"[отдали, другому, человеку, получила, посылку,..."
2,"[ужасная, синтетика, тонкая, общего, представл..."
3,"[пришел, продлил, защиту, моего, согласия, про..."
4,"[кофточка, голая, синтетика, носить, возможно, ]"


In [32]:
#Можно выполнить при помощи NLTK. Заодно проведем лемматизацию
from pymorphy2 import MorphAnalyzer
from nltk import sent_tokenize, word_tokenize, regexp_tokenize

def tokenize_lemmas(sent, pat=r"(?u)\b\w\w+\b", morph=MorphAnalyzer()):
    return [morph.parse(tok)[0].normal_form 
            for tok in regexp_tokenize(sent, pat)]
df["text_lemm"] = df["text_rare"].map(lambda x: " ".join(tokenize_lemmas(x)))
df[['text_lemm']].head()

TypeError: expected string or bytes-like object

In [46]:
#Исходя из цели автоматической обработки текста, выберем необходимые варианты предобработки текста. 
#Например, для классификации можно воспользоваться нижним регистром, удалением  пунктуации, ссылок, стоп-слов, 
#произвести лемматизацию и токенизацию
##ИЛИ воспользоваться только удалением ссылок. Т.к. это актуальнее для НС, а выбор предобработки сделал для Машинного обученя
df['text_processed']  = df['review'].str.lower()
df['text_processed'] = df['text_processed'].str.replace('\d+', '') 
df['text_processed'] = df['text_processed''].str.replace('[^\w\s]','')
df['text_processed'] = df['text_processed'].apply(stopwords)
df['text_processed'] = df['text_processed'].apply(emoji)
df['text_processed'] = df['text_processed'].apply(remove_urls)
df['text_processed'] = df['text_processed'].apply(html)
df['text_processed'] = df['text_processed'].map(lambda x: " ".join(tokenize_lemmas(x)))
df[['text_processed']].head()

,text_ready
0,качество плохой пошив ужасный горловина напере...
1,товар отдать другой человек получить посылка л...
2,ужасный синтетик тонкий общий представить карт...
3,товар прийти продавец продлить защита мой согл...
4,кофточка голый синтетик носить возможно


In [ ]:
##Пример препобученной модели HuggingFace Torch

In [17]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [23]:
#txt = df.iloc[2,2]
txt = "Хорошая погода! Наконец-то тепло!"
print(txt)

Хорошая погода! Наконец-то тепло!


In [24]:
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [25]:
 predict(txt)

array([1], dtype=int64)